In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
from matplotlib import pyplot as plt
import GPy
import climin
import time
from sklearn.utils import shuffle
from sklearn.cluster import KMeans

def smse(Y, Ystar):
    return np.mean((Y-Ystar)**2/Y.var())

def mae(Y, Ystar):
    return np.mean(np.abs(Y-Ystar))


This dataset is composed of a range of biomedical voice measurements from 42 
people with early-stage Parkinson's disease recruited to a six-month trial of 
a telemonitoring device for remote symptom progression monitoring. The 
recordings were automatically captured in the patient's homes.

Columns in the table contain subject number, subject age, subject gender, 
time interval from baseline recruitment date, motor UPDRS, total UPDRS, and 
16 biomedical voice measures. Each row corresponds to one of 5,875 voice 
recording from these individuals. The main aim of the data is to predict the 
motor and total UPDRS scores ('motor_UPDRS' and 'total_UPDRS') from the 16 
voice measures.

The data is in ASCII CSV format. The rows of the CSV file contain an instance 
corresponding to one voice recording. There are around 200 recordings per 
patient, the subject number of the patient is identified in the first column. 
For further information or to pass on comments, please contact Athanasios 
Tsanas (tsanasthanasis '@' gmail.com) or Max Little (littlem '@' 
physics.ox.ac.uk).



ATTRIBUTE INFORMATION:

subject# - Integer that uniquely identifies each subject

age - Subject age

sex - Subject gender '0' - male, '1' - female

test_time - Time since recruitment into the trial. The integer part is the 
number of days since recruitment.

** motor_UPDRS - Clinician's motor UPDRS score, linearly interpolated **

** total_UPDRS - Clinician's total UPDRS score, linearly interpolated **

Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP - Several measures of 
variation in fundamental frequency

Shimmer,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA - 
Several measures of variation in amplitude

NHR,HNR - Two measures of ratio of noise to tonal components in the voice

RPDE - A nonlinear dynamical complexity measure

DFA - Signal fractal scaling exponent

PPE - A nonlinear measure of fundamental frequency variation 


Athanasios Tsanas, Max A. Little, Patrick E. McSharry, Lorraine O. Ramig (2009),
'Accurate telemonitoring of Parkinson.s disease progression by non-invasive 
speech tests',
IEEE Transactions on Biomedical Engineering (to appear).

In [2]:
xx = np.loadtxt("parkinsons_updrs.data",skiprows=1,delimiter=',')
xx = shuffle(xx, random_state=0)

Ntest = 500
Ntrain = xx.shape[0] - Ntest

X = xx[:, 6:]
y_motor = xx[:,4]
y_total = xx[:,5]



print xx.shape

(5875, 22)


In [26]:


y_motor_train = y_motor[0:Ntrain]
y_total_train = y_total[0:Ntrain]
Xtrain = X[0:Ntrain,:]
y_motor_test = y_motor[Ntrain:]
y_total_test = y_total[Ntrain:]
Xtest = X[Ntrain:,:]

y_motor_train = y_motor_train.reshape((Ntrain,1))
y_motor_train = y_motor_train - y_motor_train.mean()
y_total_train = y_total_train.reshape((Ntrain,1))
y_total_train = y_total_train - y_total_train.mean()
Xtrain = (Xtrain - Xtrain.mean(0)[None,:])/Xtrain.std(0)[None,:]

y_motor_test = y_motor_test.reshape((Ntest,1))
y_motor_test = y_motor_test - y_motor_test.mean()
y_total_test = y_total_test.reshape((Ntest,1))
y_total_test = y_total_test - y_total_test.mean()
Xtest = (Xtest - Xtest.mean(0)[None,:])/Xtest.std(0)[None,:]

print Xtrain.shape



(5375, 16)


In [5]:
# separate lengthscale for each input
kern = GPy.kern.RBF(Xtrain.shape[1], ARD = True) + GPy.kern.White(Xtrain.shape[1])
batchsize = 500
M = 750
start = time.time()
Z = KMeans(n_clusters=M).fit(Xtrain).cluster_centers_
end = time.time()
print(end - start)
m_motor = GPy.core.SVGP(Xtrain, y_motor_train, Z, kern, likelihood=GPy.likelihoods.Gaussian(), batchsize=batchsize)
opt = climin.Adadelta(m_motor.optimizer_array, m_motor.stochastic_grad, step_rate=0.1, momentum=0.9)

patience = 0
min_log_lik = float("-inf")
min_error = float("inf")

def callback(i):
    global best_model
    global min_error
    global patience
    #t.value = str(m.log_likelihood())
    #Stop after 5000 iterations
    if i['n_iter'] > 1250 or patience > 500:
        return True
    if i['n_iter'] % 100 == 0:
        print(i['n_iter'])
        y_motor_test_pred = m_motor.predict(Xtest)
        current_error = mae(y_motor_test, y_motor_test_pred[0])
        if current_error < min_error:
            min_error = current_error
            print min_error
            print m_motor.log_likelihood()
            best_model = m_motor
    return False

start = time.time()
info = opt.minimize_until(callback)
end = time.time()
print(end - start)

y_motor_test_pred = best_model.predict(Xtest)



print "final result"
print best.model
print mae(y_motor_test, y_motor_test_pred[0])

### Exact Sparse GP

m_motor_exact = GPy.models.SparseGPRegression(Xtrain, y_motor_train, Z=Z, kernel=kern)

start = time.time()
m_motor_exact.optimize()
end = time.time()
print(end - start)

y_test_pred = m_motor_exact.predict(Xtest)



print "final result"
print m_motor_exact
print mae(ytest, y_test_pred[0])

# Full GP

m = GPy.models.GPRegression(Xtrain, y_motor_train, kernel=kern)
start = time.time()
m.optimize()
end = time.time()
print(end - start)
y_test_pred = m.predict(Xtest)

print "final result"
print m
print mae(y_motor_test, y_test_pred[0]) 

KeyboardInterrupt: 

In [ ]:
# separate lengthscale for each input
kern = GPy.kern.RBF(Xtrain.shape[1], ARD = True) + GPy.kern.White(Xtrain.shape[1])
batchsize = 500
#M = 750
#Z = KMeans(n_clusters=M).fit(Xtrain).cluster_centers_
m_total = GPy.core.SVGP(Xtrain, y_total_train, Z, kern, likelihood=GPy.likelihoods.Gaussian(), batchsize=batchsize)
opt = climin.Adadelta(m_total.optimizer_array, m_total.stochastic_grad, step_rate=0.1, momentum=0.9)

patience = 0
min_log_lik = float("-inf")
min_error = float("inf")

bet_total_model = m_total

def callback(i):
    global best_total_model
    global min_error
    global patience
    #t.value = str(m.log_likelihood())
    #Stop after 5000 iterations
    if i['n_iter'] > 1250 or patience > 500:
        return True
    if i['n_iter'] % 100 == 0:
        print(i['n_iter'])
        y_total_test_pred = m_total.predict(Xtest)
        current_error = mae(y_total_test, y_total_test_pred[0])
        if current_error < min_error:
            min_error = current_error
            print min_error
            print m_total.log_likelihood()
            best_total_model = m_total
    return False

start = time.time()
info = opt.minimize_until(callback)
end = time.time()
print(end - start)

y_total_test_pred = best_total_model.predict(Xtest)



print "final result"
print best_model
print mae(y_total_test, y_total_test_pred[0])

### Exact Sparse GP

m_total_exact = GPy.models.SparseGPRegression(Xtrain, y_total_train, Z=Z, kernel=kern)

start = time.time()
m_total_exact.optimize()
end = time.time()
print(end - start)

y_total_test_pred = m_total_exact.predict(Xtest)



print "final result"
print m_total_exact
print mae(y_total_test, y_total_test_pred[0])


m = GPy.models.GPRegression(Xtrain, y_total_train, kernel=kern)
start = time.time()
m.optimize()
end = time.time()
print(end - start)
y_test_pred = m.predict(Xtest)

print "final result"
print m
print mae(y_total_test, y_test_pred[0]) 

#m.optimize()
print m

 It is true that the
nearly 6,000 samples come from 42 patients which could lead
to some dependence between the samples, dependence that
might affect the reliability of the cross-validation. However,
only a small number of patients were recruited to the study,
and any patient-specific cross-validation is not really reliable:
there is not enough hold-out data and in our own experimental
computations the standard deviation of the errors was too
large. Therefore, simple patient-specific cross-validation is too
unstable to form a reliable estimate of the asymptotic out-ofsample
prediction error. 

http://people.maths.ox.ac.uk/tsanas/Preprints/TBME2010.pdf

# SARCOS

In [29]:
import scipy.io
train = scipy.io.loadmat('data.mat')
sarcos_inv = train['sarcos_inv']
sarcos_inv = shuffle(sarcos_inv, random_state=0)
sarcos_inv_X = sarcos_inv[:,0:21]
sarcos_inv_y = sarcos_inv[:,21]
sarcos_inv_y = sarcos_inv_y.reshape(sarcos_inv_y.shape[0], 1)
print sarcos_inv.shape

test = scipy.io.loadmat('sarcos_inv_test.mat')
sarcos_inv_test = test['sarcos_inv_test']

sarcos_inv_test_X = sarcos_inv_test[:,0:21]
sarcos_inv_test_y = sarcos_inv_test[:,21]
sarcos_inv_test_y = sarcos_inv_test_y.reshape(sarcos_inv_test_y.shape[0], 1)
print sarcos_inv_test.shape

sarcos_inv_y_not_scaled = sarcos_inv_y
sarcos_inv_X_not_scaled = sarcos_inv_X

sarcos_inv_y = (sarcos_inv_y - sarcos_inv_y.mean())#/sarcos_inv_y_not_scaled.std()
sarcos_inv_X = (sarcos_inv_X - sarcos_inv_X.mean(0)[None,:])/sarcos_inv_X.std(0)[None,:]

sarcos_inv_test_y = (sarcos_inv_test_y - sarcos_inv_test_y.mean())#/sarcos_inv_y_not_scaled.std()
sarcos_inv_test_X = (sarcos_inv_test_X - sarcos_inv_test_X.mean(0)[None,:])/sarcos_inv_test_X.std(0)[None,:]

(44484, 28)
(4449, 28)


In [80]:
def smse(Y, Ystar):
    return np.mean((Y-Ystar)**2/Y.var())


# separate lengthscale for each input
kern = GPy.kern.RBF(sarcos_inv_X.shape[1], ARD = True) + GPy.kern.White(sarcos_inv_X.shape[1])
# + GPy.kern.Bias(sarcos_inv_X.shape[1]) 
batchsize = 500
M = 1250
#Z = np.random.permutation(sarcos_inv_X)[:M]
start = time.time()
#Z = KMeans(n_clusters=M).fit(sarcos_inv_X).cluster_centers_
end = time.time()
print(end - start)

m_svgp = GPy.core.SVGP(sarcos_inv_X, sarcos_inv_y, Z, kern, likelihood=GPy.likelihoods.Gaussian(), batchsize=batchsize)



3.69548797607e-05


In [87]:
opt = climin.Adadelta(m_svgp.optimizer_array, m_svgp.stochastic_grad, step_rate=0.0015, decay=0.75, momentum=0.9)

patience = 0
min_log_lik = float("-inf")
min_error = float("inf")

In [88]:
def callback(i):
    global best_model
    global min_error
    #t.value = str(m.log_likelihood())
    #Stop after 5000 iterations
    if i['n_iter'] > 5000 or patience > 500:
        return True
    if i['n_iter'] % 100 == 0:
        print(i['n_iter'])
        y_test_pred = m_svgp.predict(sarcos_inv_test_X)
        current_error = smse(sarcos_inv_test_y, y_test_pred[0])
        if current_error < min_error:
            min_error = current_error
            print min_error
            print m_svgp.log_likelihood()
            best_model = m_svgp
    return False

start = time.time()
info = opt.minimize_until(callback)
end = time.time()
print(end - start)

y_test_pred = best_model.predict(sarcos_inv_test_X)



print "final result"
print best_model
print smse(sarcos_inv_test_y, y_test_pred[0])

100
0.0183249884367
-116289.937133
200
0.0183023327809
-115865.319083
300
0.0182666636354
-115674.67663
400
500
0.0182456272482
-116848.860286
600
0.0182380914868
-113904.696761
700
0.0182201931157
-116915.825087
800
0.0181973990322
-114994.400799
900
0.0181881810838
-118709.100716
1000
0.0181725375005
-115415.209702
1100
1200
0.0181505728839
-115225.479418
1300
1400
1500
0.0181489119473
-121274.66912
1600
0.0181390940334
-114188.866645
1700
1800
0.0181265365253
-114407.413872
1900
0.0181240960768
-119903.958211
2000
0.0181135056483
-111383.403574
2100
0.0181016600762
-118791.117971
2200


KeyboardInterrupt: 

In [55]:
m = GPy.models.SparseGPRegression(sarcos_inv_X, sarcos_inv_y, Z=Z, kernel=kern)

start = time.time()
m.optimize()
end = time.time()
print(end - start)

y_test_pred = m.predict(sarcos_inv_test_X)



print "final result"
print m
print smse(sarcos_inv_test_y, y_test_pred[0])

37220.393538
final result

Name                              : sparse gp
Log-likelihood                    : -105688.103879
Number of Parameters              : 26274
Number of Optimization Parameters : 26274
Updates                           : True
Parameters:
  sparse_gp.               |        Value        |  Constraint  |  Prior  |  Tied to
  inducing inputs          |         (1250, 21)  |              |         |         
  sum.rbf.variance         |      142.958525486  |     +ve      |         |         
  sum.rbf.lengthscale      |              (21,)  |     +ve      |         |         
  sum.white.variance       |  3.39091011839e-06  |     +ve      |         |         
  Gaussian_noise.variance  |        5.307253342  |     +ve      |         |         
0.0104279232838


In [7]:
kern = GPy.kern.RBF(sarcos_inv_X.shape[1], ARD = True) + GPy.kern.White(sarcos_inv_X.shape[1])
#+ GPy.kern.Bias(sarcos_inv_X.shape[1]) 
M = 4096

m = GPy.models.GPRegression(sarcos_inv_X[:M], sarcos_inv_y[:M], kernel=kern)
start = time.time()
m.optimize()
end = time.time()
print(end - start)
y_test_pred = m.predict(sarcos_inv_test_X)



print "final result"
print m
print smse(sarcos_inv_test_y, y_test_pred[0])

429.651548862
final result

Name                              : GP regression
Log-likelihood                    : 1321.94736851
Number of Parameters              : 5
Number of Optimization Parameters : 5
Updates                           : True
Parameters:
  GP_regression.           |        Value        |  Constraint  |  Prior  |  Tied to
  sum.rbf.variance         |      2.39861132239  |     +ve      |         |         
  sum.rbf.lengthscale      |      4.80105642595  |     +ve      |         |         
  sum.bias.variance        |  0.000120010229428  |     +ve      |         |         
  sum.white.variance       |   0.00757564574291  |     +ve      |         |         
  Gaussian_noise.variance  |   0.00757564574291  |     +ve      |         |         
0.0221649759547


https://openann.github.io/OpenANN-apidoc/SARCOSBenchmark.html